# CatBoost Classifier

## Part 1 - Data Preprocessing

### Importing the dataset

In [30]:
import pandas as pd
dataset = pd.read_csv('churn_modelling.csv')

In [ ]:
dataset.head()

### Checking missing data

In [ ]:
dataset.info()

### Handling categorical variables

CustomerId and Surname columns

In [33]:
dataset.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)

In [ ]:
dataset.head()

Geography column

In [ ]:
dataset['Geography'].unique()

In [36]:
geography_dummies = pd.get_dummies(dataset['Geography'], drop_first = True)

In [ ]:
geography_dummies

In [38]:
dataset = pd.concat([geography_dummies, dataset], axis = 1)

In [ ]:
dataset.head()

In [40]:
dataset.drop(['Geography'], axis = 1, inplace = True)

In [ ]:
dataset.head()

Gender column

In [ ]:
dataset['Gender'].unique()

In [43]:
dataset['Gender'] = dataset['Gender'].apply(lambda x: 0 if x == 'Female' else 1)

In [ ]:
dataset.head()

### Creating the Training Set and the Test Set

Getting the inputs and output

In [45]:
X = dataset.iloc[:, :-1].values

In [46]:
y = dataset.iloc[:, -1].values

In [ ]:
X

In [ ]:
y

Getting the Training Set and the Test Set

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Part 2 - Building and training the model

### Building the model

In [50]:
#!pip install catboost

In [51]:
import catboost as cb
model = cb.CatBoostClassifier()

### Training the model

In [ ]:
model.fit(X_train, y_train)

### Inference

In [53]:
y_pred = model.predict(X_test)

### Predicting the result of a single observation

**Homework**

Use our model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [ ]:
print(model.predict([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

Therefore, our model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "0, 0" in the first two columns. That's because of course the predict method expects the dummy values of the Geography variable.

## Part 3: Evaluating the model

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Extract true positives, false positives, true negatives, and false negatives
tn, fp, fn, tp = cm.ravel()

# Print the confusion matrix and its components
print("Confusion Matrix:")
print(cm)
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")

### Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

### k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model,
                             X = X,
                             y = y,
                             scoring = 'accuracy',
                             cv = 10)
print(f"Accuracy: {accuracies.mean()*100} %")
print(f"Standard Deviation: {accuracies.std()*100} %")

## Grid Search (This process may take a significant amount of time to complete.)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'learning_rate': [0.001,0.005,0.01], 'depth': [4,7,10], 'l2_leaf_reg': [2,6,10], 'random_strength': [0,5,10]}]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search.fit(X, y)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(f"Best Accuracy: {best_accuracy*100} %")
print("Best Parameters:", best_parameters)

In [ ]:
# Explanation of each parameter in the grid search:
# 'learning_rate': This parameter controls the step size at each iteration while moving toward a minimum of a loss function. 
#                  Lower values make the model more robust to overfitting but require more iterations to converge.
# 'depth': This parameter specifies the depth of the tree. Deeper trees can model more complex patterns but may lead to overfitting.
# 'l2_leaf_reg': This parameter is the L2 regularization term on weights. It helps to avoid overfitting by penalizing large weights.
# 'random_strength': This parameter adds randomness to the model, which can help in reducing overfitting by making the model more robust.
